# Data Cleaning

This notebook includes data cleaning for 2 different types of data used for EDA and Modeling. The first combines Major Incidents files, which will also be connected to Tableau and joined with a shapefile. The cost and loss data from this source will also be used to build predictions in the modeling portion. The second is focused on acres burned since 1950, which will be used for the time series model.

In [1]:
# import libraries
import pandas as pd
import numpy as np

from uszipcode import Zipcode, SearchEngine

import warnings
warnings.filterwarnings("ignore")

### Import major incidents files

These are separated into Northern and Southern California Area Coordination Centers. Each report had to be downloaded separately by year. I will pull them all into individual variables which can then be concatenated together to create one inclusive incident report dataframe.

This data includes all major wildfire incidents, defined as larger than 100 acres.

In [2]:
fire_df_nc_19 = pd.read_excel('../01_data_collection/claire_data/Significant Incident Summary Spreadsheet by GACC norcal 2019.xlsx')
fire_df_nc_18 = pd.read_excel('../01_data_collection/claire_data/Significant Incident Summary Spreadsheet by GACC norcal 2018.xlsx')
fire_df_nc_17 = pd.read_excel('../01_data_collection/claire_data/Significant Incident Summary Spreadsheet by GACC norcal 2017.xlsx')
fire_df_nc_16 = pd.read_excel('../01_data_collection/claire_data/Significant Incident Summary Spreadsheet by GACC norcal 2016.xlsx')
fire_df_nc_15 = pd.read_excel('../01_data_collection/claire_data/Significant Incident Summary Spreadsheet by GACC norcal 2015.xlsx')
fire_df_nc_14 = pd.read_excel('../01_data_collection/claire_data/Significant Incident Summary Spreadsheet by GACC norcal 2014.xlsx')

fire_df_sc_19 = pd.read_excel('../01_data_collection/claire_data/Significant Incident Summary Spreadsheet by GACC socal 2019.xlsx')
fire_df_sc_18 = pd.read_excel('../01_data_collection/claire_data/Significant Incident Summary Spreadsheet by GACC socal 2018.xlsx')
fire_df_sc_17 = pd.read_excel('../01_data_collection/claire_data/Significant Incident Summary Spreadsheet by GACC socal 2017.xlsx')
fire_df_sc_16 = pd.read_excel('../01_data_collection/claire_data/Significant Incident Summary Spreadsheet by GACC socal 2016.xlsx')
fire_df_sc_15 = pd.read_excel('../01_data_collection/claire_data/Significant Incident Summary Spreadsheet by GACC socal 2015.xlsx')
fire_df_sc_14 = pd.read_excel('../01_data_collection/claire_data/Significant Incident Summary Spreadsheet by GACC socal 2014.xlsx')

In [3]:
fires_df = pd.concat([fire_df_nc_14, fire_df_sc_14, 
                      fire_df_nc_15, fire_df_sc_15,
                      fire_df_nc_16, fire_df_sc_16,
                      fire_df_nc_17, fire_df_sc_17,
                      fire_df_nc_18, fire_df_sc_18, 
                      fire_df_nc_19, fire_df_sc_19]).reset_index(drop=True)

In [4]:
fires_df.head(2)

,State-Unit,Incident Number,Incident Name,Incident Type,Start Date,IC Name,Team Type,Latitude,Longitude,Size,Unit of Measure,Costs,Anticipated Containment / Completion Date,Structures Destroyed
0,CA-LNU,CA-LNU-0005333,BUTTS,WF,07/01/2014 1405 CT,"DC, SCOTT UPTON, LNU-CALFIRE",Type 3 IC,39,123,3200.0,Acres,6500000.0,NaN,5
1,CA-AEU,CA-AEU-018237,SAND,WF,07/25/2014 1834 CT,"BC Mike Olivarria, AEU",Type 3 IC,39,121,4240.0,Acres,875500.0,08/02/2014 1400,67


In [5]:
fires_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 588 entries, 0 to 587
Data columns (total 14 columns):
 #   Column                                     Non-Null Count  Dtype  
---  ------                                     --------------  -----  
 0   State-Unit                                 588 non-null    object 
 1   Incident Number                            588 non-null    object 
 2   Incident Name                              588 non-null    object 
 3   Incident Type                              588 non-null    object 
 4   Start Date                                 588 non-null    object 
 5   IC Name                                    587 non-null    object 
 6   Team Type                                  559 non-null    object 
 7   Latitude                                   588 non-null    int64  
 8   Longitude                                  588 non-null    int64  
 9   Size                                       588 non-null    float64
 10  Unit of Measure           

### Data Cleaning

There are several features to be updated here. A few of them are:
- Making all incident names uppercase to match the shapefile data
- Year: extract year from start date
- Costs: remove all fires that don't have an associated cost
- Incident Type: remove all incidents that are not classified as wildfires
- State: remove all fires outside of CA (a few fires in HI are under Southern CA's jurisdiction)
- County: use 'uszipcode' library to extract county using latitude and longitude measures

In [6]:
fires_df['Fire Name'] = fires_df['Incident Name'].str.upper()

In [7]:
fires_df['start_date'] = pd.to_datetime(fires_df['Start Date'], utc=True).dt.date

In [8]:
fires_df['year'] = pd.to_datetime(fires_df['Start Date'], utc=True).dt.year

In [9]:
fires_df.head(2)

,State-Unit,Incident Number,Incident Name,Incident Type,Start Date,IC Name,Team Type,Latitude,Longitude,Size,Unit of Measure,Costs,Anticipated Containment / Completion Date,Structures Destroyed,Fire Name,start_date,year
0,CA-LNU,CA-LNU-0005333,BUTTS,WF,07/01/2014 1405 CT,"DC, SCOTT UPTON, LNU-CALFIRE",Type 3 IC,39,123,3200.0,Acres,6500000.0,NaN,5,BUTTS,2014-07-01,2014
1,CA-AEU,CA-AEU-018237,SAND,WF,07/25/2014 1834 CT,"BC Mike Olivarria, AEU",Type 3 IC,39,121,4240.0,Acres,875500.0,08/02/2014 1400,67,SAND,2014-07-25,2014


In [10]:
fires_df.dropna(subset=['Costs'], axis=0, inplace=True)

In [11]:
fires_df['Incident Type'].value_counts()

WF    573
CX      9
OT      2
WS      1
LS      1
FL      1
Name: Incident Type, dtype: int64

In [12]:
# Drop all non-wildfire incidents
fires_df = fires_df[fires_df['Incident Type'] == 'WF']

In [13]:
fires_df['State-Unit'].unique()

array(['CA-LNU', 'CA-AEU', 'CA-NEU', 'CA-NOD', 'CA-ENF', 'CA-SHU',
       'CA-SKU', 'CA-MDF', 'CA-KNF', 'CA-HIA', 'CA-LMU', 'CA-BTU',
       'CA-LNF', 'CA-SWR', 'CA-SHF', 'CA-MEU', 'CA-MCP', 'CA-ANF',
       'CA-TCU', 'CA-BDF', 'CA-INF', 'CA-STF', 'CA-BDU', 'CA-FKU',
       'CA-CND', 'CA-MMU', 'CA-YNP', 'CA-MVU', 'CA-LAC', 'CA-FHL',
       'CA-CBD', 'CA-CNF', 'CA-SQF', 'CA-SNF', 'CA-RRU', 'CA-SBC',
       'CA-BEU', 'CA--BRR', 'CA-SRF', 'CA-PNF', 'CA-HUU', 'HI-CNTY',
       'CA-SCU', 'HI-JCR', 'CA-VNC', 'CA-KRN', 'CA-OVD', 'CA-LPF',
       'CA-ORC', 'CA-SLU', 'CA-TUU', 'CA-KNP', 'CA-TGU', 'CA-AFV',
       'CA-MNF', 'CA-CZU', 'HI-DFW', 'CA-MRN', 'CA-TNF', 'CA-LFD',
       'HI-HVP', 'CA-WNP', 'CA-CNP', 'CA-CDD'], dtype=object)

In [14]:
fires_df = fires_df[fires_df['State-Unit'].str.startswith('CA')]

In [15]:
fires_df.reset_index(drop=True, inplace=True)

In [16]:
# Code adapted from Emiko Sano

def county_name(lat, long, radius=100):
    '''Takes in latitude and longitude (as an iterable) and returns the closest county name for the given coordinates'''
    lat, long = lat, long * -1
    search = SearchEngine()
    result = search.by_coordinates(lat, long)
    # get the county name
    try:
        county = result[0].county
        # this is in 'XXXX County', so fix the format so that it's only the county name without 'County' at the end
        county = county.split()
        # get everything but the last item (which is 'County')
        county = county[:-1]
        # return a string
        return ' '.join(county)
    except:
        print(f'Something went wrong. Check your coordinates: {lat, long}') 

In [17]:
def county_column(df):
    '''Applies the county_name function to latitude and longitude columns in order to create a new county column'''
    county_list = []
    for num in range(len(df)):
        latitude, longitude = float(fires_df.iloc[num, 7]), float(fires_df.iloc[num, 8])
        county_list.append(county_name(latitude, longitude))
        
    return county_list

In [18]:
fires_df['county'] = county_column(fires_df)

Something went wrong. Check your coordinates: (33.0, -118.0)
Something went wrong. Check your coordinates: (34.0, -125.0)
Something went wrong. Check your coordinates: (33.0, -118.0)
Something went wrong. Check your coordinates: (33.0, -118.0)


In [19]:
fires_df[fires_df['county'].isna()]

,State-Unit,Incident Number,Incident Name,Incident Type,Start Date,IC Name,Team Type,Latitude,Longitude,Size,Unit of Measure,Costs,Anticipated Containment / Completion Date,Structures Destroyed,Fire Name,start_date,year,county
45,CA-MCP,CA-MCP-001359,Combat,WF,05/16/2014 1115 CT,None Assigned,NaN,33,118,300.0,Acres,10000.0,NaN,0,COMBAT,2014-05-16,2014,None
61,CA-ANF,CA-ANF-003975,Shoemaker,WF,08/14/2014 2348 CT,A. LEYVA,Type 3 IC,34,125,190.0,Acres,700000.0,NaN,0,SHOEMAKER,2014-08-14,2014,None
136,CA-MCP,CA-MCP-001207,CARL,WF,04/28/2015 1415 PDT,MCP,NaN,33,118,4000.0,Acres,1000.0,NaN,0,CARL,2015-04-28,2015,None
328,CA-MCP,CA-MCP-17-2047,Cristianitos,WF,06/28/2017 1630 PDT,John Crook MCP\nKen Cruz ORC,Type 3 IC,33,118,761.0,Acres,500000.0,07/01/2017 1200,0,CRISTIANITOS,2017-06-28,2017,None


In [20]:
# Manually correcting counties that had NaN values
fires_df.loc[45, 'county'] = 'San Diego'
fires_df.loc[61, 'county'] = 'Los Angeles'
fires_df.loc[136, 'county'] = 'San Diego'
fires_df.loc[328, 'county'] = 'San Diego'

In [21]:
fires_df_clean = fires_df.drop(columns = ['State-Unit', 'Start Date', 
                               'IC Name', 'Team Type', 'Anticipated Containment / Completion Date'])
fires_df_clean.head(2)

,Incident Number,Incident Name,Incident Type,Latitude,Longitude,Size,Unit of Measure,Costs,Structures Destroyed,Fire Name,start_date,year,county
0,CA-LNU-0005333,BUTTS,WF,39,123,3200.0,Acres,6500000.0,5,BUTTS,2014-07-01,2014,Lake
1,CA-AEU-018237,SAND,WF,39,121,4240.0,Acres,875500.0,67,SAND,2014-07-25,2014,Placer


In [22]:
fires_df_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 559 entries, 0 to 558
Data columns (total 13 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Incident Number       559 non-null    object 
 1   Incident Name         559 non-null    object 
 2   Incident Type         559 non-null    object 
 3   Latitude              559 non-null    int64  
 4   Longitude             559 non-null    int64  
 5   Size                  559 non-null    float64
 6   Unit of Measure       559 non-null    object 
 7   Costs                 559 non-null    float64
 8   Structures Destroyed  559 non-null    int64  
 9   Fire Name             559 non-null    object 
 10  start_date            559 non-null    object 
 11  year                  559 non-null    int64  
 12  county                559 non-null    object 
dtypes: float64(2), int64(4), object(7)
memory usage: 56.9+ KB


In [23]:
fires_df_clean.describe()

,Latitude,Longitude,Size,Costs,Structures Destroyed,year
count,559.000000,559.000000,559.000000,5.590000e+02,559.000000,559.000000
mean,37.402504,120.245081,9640.852594,9.073276e+06,73.733453,2016.515206
std,2.727280,2.023794,31933.767732,2.518517e+07,861.440021,1.612908
min,33.000000,116.000000,1.000000,0.000000e+00,0.000000,2014.000000
25%,35.000000,119.000000,319.500000,3.500000e+05,0.000000,2015.000000
50%,37.000000,120.000000,1021.000000,1.500000e+06,0.000000,2017.000000
75%,40.000000,122.000000,4305.000000,6.456838e+06,2.000000,2018.000000
max,42.000000,125.000000,410203.000000,2.625000e+08,18804.000000,2019.000000


In [24]:
fires_df_clean = fires_df_clean[(fires_df_clean['Size'] > 100) & (fires_df_clean['Costs'] > 0)]

In [25]:
fires_df_clean['cost_per_acre'] = fires_df_clean['Costs'] / fires_df_clean['Size']

In [26]:
fires_df_clean['structures_per_acre'] = fires_df_clean['Structures Destroyed'] / fires_df_clean['Size']

In [27]:
fires_df_clean.to_csv('../01_data_collection/claire_data/fires_compiled.csv', index=False)

### Acres Burned Over Time

This will be used for time series forecasting, and so the only 2 measures that are important to us are Year and Acres burned.

In [28]:
acres_df = pd.read_csv('../01_data_collection/claire_data/California_Fire_Perimeters.csv')

In [29]:
acres_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12202 entries, 0 to 12201
Data columns (total 19 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   OBJECTID      12202 non-null  int64  
 1   YEAR_         12140 non-null  float64
 2   STATE         12148 non-null  object 
 3   AGENCY        12202 non-null  object 
 4   UNIT_ID       12199 non-null  object 
 5   FIRE_NAME     12174 non-null  object 
 6   INC_NUM       11720 non-null  object 
 7   ALARM_DATE    8214 non-null   object 
 8   CONT_DATE     3375 non-null   object 
 9   CAUSE         12190 non-null  float64
 10  COMMENTS      1886 non-null   object 
 11  REPORT_AC     3836 non-null   float64
 12  GIS_ACRES     12202 non-null  float64
 13  C_METHOD      3759 non-null   float64
 14  OBJECTIVE     12146 non-null  float64
 15  FIRE_NUM      10952 non-null  object 
 16  GlobalID      12202 non-null  object 
 17  SHAPE_Length  12202 non-null  float64
 18  SHAPE_Area    12202 non-nu

In [30]:
acres_df.dropna(subset=['YEAR_'], inplace=True)

In [31]:
acres_df = acres_df.sort_values(by='YEAR_')

In [32]:
acres_df = acres_df[(acres_df['YEAR_'] >= 1950) & (acres_df['GIS_ACRES'] > 100)]

In [33]:
acres_df = acres_df[['YEAR_', 'GIS_ACRES']]

In [34]:
acres_df.columns = ['year', 'acres_burned']

In [35]:
acres_df['year'] = acres_df['year'].astype('int64')

In [36]:
acres_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8479 entries, 4941 to 12201
Data columns (total 2 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   year          8479 non-null   int64  
 1   acres_burned  8479 non-null   float64
dtypes: float64(1), int64(1)
memory usage: 198.7 KB


In [37]:
acres_df = acres_df.groupby('year').sum()

In [38]:
acres_df.head()

,acres_burned
year,
1950,537538.72271
1951,305721.20676
1952,132370.47070
1953,333854.81594
1954,218733.99078


In [39]:
# Manually adding 2020, acre count here: https://www.fire.ca.gov/incidents/2020/
acres_df = acres_df.append(pd.DataFrame({'acres_burned': 4149345}, index=[2020]))

In [40]:
acres_df.tail(6)

,acres_burned
2015,7.847164e+05
2016,5.402921e+05
2017,1.415352e+06
2018,1.584802e+06
2019,2.753229e+05
2020,4.149345e+06


In [41]:
acres_df.to_csv('../01_data_collection/claire_data/acres_clean.csv')